##Instalar todas las dependencias y configurar el entorno

In [1]:
#!apt-get install python-dev python-snappy

In [2]:
pip install tensorflow-data-validation==0.24.0

     |████████████████████████████████| 1.3MB 17.3MB/s 
     |████████████████████████████████| 1.8MB 54.4MB/s 
     |████████████████████████████████| 296kB 54.7MB/s 
     |████████████████████████████████| 51kB 9.5MB/s 
     |████████████████████████████████| 63.8MB 42kB/s 
     |████████████████████████████████| 378kB 50.5MB/s 
     |████████████████████████████████| 9.0MB 46.6MB/s 
     |████████████████████████████████| 61kB 10.8MB/s 
     |████████████████████████████████| 51kB 9.7MB/s 
     |████████████████████████████████| 61kB 9.0MB/s 
     |████████████████████████████████| 829kB 45.7MB/s 
     |████████████████████████████████| 2.0MB 40.6MB/s 
     |████████████████████████████████| 153kB 59.1MB/s 
     |████████████████████████████████| 174kB 55.6MB/s 
     |████████████████████████████████| 174kB 58.4MB/s 
     |████████████████████████████████| 92kB 12.9MB/s 
     |████████████████████████████████| 122kB 57.3MB/s 
     |████████████████████████████████| 184kB 59.0MB/s 
 

In [1]:
import sys

# Confirm that we're using Python 3
assert sys.version_info.major is 3, 'Oops, not running Python 3. Use Runtime > Change runtime type'

## Importar las librerías para el proyecto

In [3]:
import pandas as pd
import tensorflow as tf
import tensorflow_data_validation as tfdv

print("versión de tensorflow: {}\nversión de data_validation: {} ".format(tf.__version__,tfdv.__version__))

#from __future__ import print_function

versión de tensorflow: 2.4.0
versión de data_validation: 0.24.0 


## Análisis descriptivo del dataset


In [5]:
dataset = pd.read_csv("datos/pollution-small.csv")

In [6]:
dataset.head()

,Date,pm10,no2,so2,soot
0,1/1/2009,98.67,14.10,44.38,34.81
1,1/2/2009,52.33,14.10,29.75,33.06
2,1/3/2009,74.67,20.50,36.25,39.25
3,1/4/2009,72.00,17.30,46.44,34.38
4,1/5/2009,81.00,25.64,56.56,45.59


In [7]:
dataset.shape

(2188, 5)

In [8]:
training_data = dataset[:1600]

In [9]:
training_data.describe()

,pm10,no2,so2,soot
count,1600.000000,1600.000000,1600.000000,1600.000000
mean,49.656494,30.980519,16.229981,21.551956
std,35.211906,12.400788,10.621896,12.127354
min,6.380000,9.740000,4.010000,6.000000
25%,28.345000,22.567500,9.777500,14.400000
50%,38.835000,28.715000,13.275000,18.630000
75%,58.050000,36.370000,19.282500,24.072500
max,277.250000,138.010000,123.130000,107.650000


In [10]:
test_set = dataset[1600:]

In [11]:
test_set.describe()

,pm10,no2,so2,soot
count,588.000000,588.000000,588.000000,588.000000
mean,44.648248,37.296922,13.605170,18.441310
std,28.992087,10.940050,5.098944,6.596459
min,11.900000,15.070000,4.990000,8.000000
25%,28.337500,29.217500,10.122500,14.410000
50%,35.555000,35.815000,12.345000,17.090000
75%,50.812500,43.872500,15.855000,20.962500
max,273.770000,106.030000,38.030000,87.210000


##Análisis descriptivo y validación con TFDV

### Generar análisis descriptivo del dataset


In [12]:
##calcular estadísticas del dataframe
train_stats = tfdv.generate_statistics_from_dataframe(dataframe=training_data)

In [13]:
train_stats

datasets {
  num_examples: 1600
  features {
    type: STRING
    string_stats {
      common_stats {
        num_non_missing: 1600
        min_num_values: 1
        max_num_values: 1
        avg_num_values: 1.0
        num_values_histogram {
          buckets {
            low_value: 1.0
            high_value: 1.0
            sample_count: 160.0
          }
          buckets {
            low_value: 1.0
            high_value: 1.0
            sample_count: 160.0
          }
          buckets {
            low_value: 1.0
            high_value: 1.0
            sample_count: 160.0
          }
          buckets {
            low_value: 1.0
            high_value: 1.0
            sample_count: 160.0
          }
          buckets {
            low_value: 1.0
            high_value: 1.0
            sample_count: 160.0
          }
          buckets {
            low_value: 1.0
            high_value: 1.0
            sample_count: 160.0
          }
          buckets {
            low_value: 

### Inferir el esquema 

In [14]:
tfdv.visualize_statistics(train_stats)

In [15]:
schema = tfdv.infer_schema(statistics=train_stats)

In [16]:
tfdv.display_schema(schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'Date',BYTES,required,,-
'pm10',FLOAT,required,,-
'no2',FLOAT,required,,-
'so2',FLOAT,required,,-
'soot',FLOAT,required,,-


### Calcular los estadísticos  descriptivos del conjunto de testing


In [17]:
test_stats = tfdv.generate_statistics_from_dataframe(dataframe=test_set)

In [18]:
tfdv.visualize_statistics(test_stats)

## Comparar estadísticos del conjunto de test con Schema

### Buscar anomalías en los nuevos datos

In [19]:
type(test_stats)

tensorflow_metadata.proto.v0.statistics_pb2.DatasetFeatureStatisticsList

In [20]:
#comparar estadísticas de un conjunto de datos con un esquema
anomalies = tfdv.validate_statistics(statistics=test_stats, schema=schema)

### Mostrar todas las anomalías detectadas

- Número entero mayor que  10
- Tipo STRING cuando se esperaba un tipo INT
- Tipo FLOAT cuando se esperaba un tipo INT
- Número entero menor que 0

In [21]:
tfdv.display_anomalies(anomalies)

### Nuevos datos CON anomalías

In [22]:
test_set_copy = test_set.copy()

In [23]:
test_set_copy.drop("soot", axis=1, inplace=True)

### Estadísticos basados en datos con anomalías

In [24]:
test_set_copy_stats = tfdv.generate_statistics_from_dataframe(dataframe=test_set_copy)

In [25]:
anomalies_new = tfdv.validate_statistics(statistics=test_set_copy_stats, schema=schema)

In [26]:
tfdv.display_anomalies(anomalies_new)

/usr/local/lib/python3.6/dist-packages/tensorflow_data_validation/utils/display_util.py:173: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


,Anomaly short description,Anomaly long description
Feature name,,
'soot',Column dropped,Column is completely missing


## Paso 5: Preparar el esquema para subir a producción

In [28]:
schema.default_environment.append("TRAINING")# trainng environment 
schema.default_environment.append("SERVING")# production environment

### Eliminar la columna objetivo del esquema de producción

In [29]:
tfdv.get_feature(schema, "soot").not_in_environment.append("SERVING")

### Comprobar anomalías entre el entorno del servidor y nuevos datos entrantes

In [30]:
serving_env_anomalies = tfdv.validate_statistics(test_set_copy_stats, schema, environment="SERVING" )

In [31]:
tfdv.display_anomalies(serving_env_anomalies)

## Paso 6: Congelar el esquema

In [32]:
tfdv.write_schema_text(schema = schema, output_path = "pollution_schema.pbtxt")